In [3]:
import sqlite3 as sql
import pandas as pd 

Apuntador = sql.connect(r'src\Back_end\DataBase\DataUsers.db')

OperationalError: unable to open database file

In [3]:
Apuntador.execute('''
                    CREATE TABLE IF NOT EXISTS 'Usuarios_Registrados' (
                        'Id' INT AUTO_INCREMENT,
                        'Nombre_Usuario' TEXT PRIMARY KEY NOT NULL,
                        'Contraseña' TEXT NOT NULL,
                        'Rol' INT NOT NULL,
                        'Email' TEXT NOT NULL,
                        CHECK(Rol IN(1,2))
                        )
                ''')

In [42]:
Apuntador.execute('''
                    CREATE TABLE IF NOT EXISTS 'Roles' (
                        'Id_Rol' INT PRIMARY KEY,
                        'Rol' TEXT NOT NULL UNIQUE,
                        CHECK(Rol IN("Estudiante", "Profesor")),
                        FOREIGN KEY(Id_Rol) REFERENCES Usuarios_Registrados(Rol)
                        )
                ''')

In [131]:
Apuntador.execute("INSERT INTO Roles (Id_Rol, Rol) VALUES ('2', 'Profesor')")

IntegrityError: CHECK constraint failed: Rol IN("Estudiante", "Profesor")

In [7]:
a = Apuntador.execute("SELECT MAX(Id) FROM Usuarios_Registrados")
a = a.fetchone()[0]
num = int(a)+1 if (a is not None) else 1
Apuntador.execute(f"INSERT INTO Usuarios_Registrados (Id, Nombre_Usuario, Contraseña, Rol, Email) VALUES ({num},'JoseJose', '12345678', 2, 'a')")

In [12]:
df = pd.read_sql_query("PRAGMA table_info(Usuarios_Registrados)", Apuntador)

In [53]:
Apuntador.execute("DELETE FROM Usuarios_Registrados")

In [7]:
Apuntador.execute("DROP TABLE Usuarios_Registrados")

In [190]:
b = Apuntador.execute("SELECT * FROM Usuarios_Registrados")
b = b.fetchall()
print(b)

[(1, 'Ariel_C', ';<=>?@AB', 1, 'noe@gmail.com')]


In [11]:
df = pd.read_sql_query("SELECT Usuarios_Registrados.Nombre_Usuario, Roles.Rol FROM Usuarios_Registrados JOIN Roles ON Usuarios_Registrados.Rol = Roles.Id_Rol", Apuntador)

In [196]:
b = Apuntador.execute("PRAGMA table_info('Usuarios_Registrados')")
b = bool(b.fetchall())
print(b)

True


In [161]:
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", Apuntador)

In [6]:
b = Apuntador.execute("SELECT name FROM sqlite_master WHERE type='table'")
b = b.fetchall()
for i in range(len(b)):
    Apuntador.execute(f"DROP TABLE '{b[i][0]}'")

#Cuidado

In [13]:
Apuntador.execute('''
                    CREATE FUNCTION 'get_max_id'(table_name TEXT)
                    RETURNS INT
                    AS
                    BEGIN
                    IF NOT EXISTS (SELECT * FROM sqlite_master WHERE tbl_name = table_name AND type = 'table');
                    THEN
                        RAISE EXCEPTION WITH MESSAGE 'Table % not found', table_name;
                    END IF;
                    SELECT name FROM pragma_table_info(table_name) WHERE pk = 1 INTO attribute_name;
                    SELECT MAX(attribute_name) FROM table_name INTO max_id;
                    RETURN max_id;
                    END;
''')

OperationalError: near "FUNCTION": syntax error

In [8]:
df = pd.read_sql_query("SELECT * FROM Usuarios_Registrados;", Apuntador)

In [10]:
df = pd.read_sql_query("SELECT get_max_id('Usuarios_Registrados');", Apuntador)

DatabaseError: Execution failed on sql 'SELECT get_max_id('Usuarios_Registrados');': no such function: get_max_id

In [9]:
df

,Id,Nombre_Usuario,Contraseña,Rol,Email
0,1,PepeSierra,12345678,2,a
1,2,JoseJose,12345678,2,a


In [2]:
Apuntador.commit()
Apuntador.close()